In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("check").getOrCreate()



In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame  # or: from pyspark.sql.dataframe import DataFrame
from pyspark.sql import functions as F, types as T
from main.utility.logging_config import logger
df: DataFrame = (
    spark.read.format("csv")
    .option("inferSchema", True)
    .option("header", True)
    .load("../fk.csv")
)
df.show()
logger.log(level=10,msg="hi")

In [1]:
import os
print("JAVA_HOME:", os.environ.get("JAVA_HOME"))

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("check").getOrCreate()
print("Spark:", spark.version)

import boto3
print("STS:", boto3.client("sts").get_caller_identity())

df = spark.read.option("header", True).option("inferSchema", True)\
       .csv("s3a://mkdv1/brands.csv")
df.show(5)

JAVA_HOME: /opt/java/openjdk


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/03 08:31:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/09/03 08:31:07 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Spark: 3.5.1
STS: {'UserId': 'AROAZI2LGOPW3CVN6PGSU:i-0399e8193fe8f80bc', 'Account': '637423481837', 'Arn': 'arn:aws:sts::637423481837:assumed-role/server/i-0399e8193fe8f80bc', 'ResponseMetadata': {'RequestId': '2bb8eccc-e8d8-4604-b477-0bed7ce5095e', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '2bb8eccc-e8d8-4604-b477-0bed7ce5095e', 'x-amz-sts-extended-request-id': 'MTp1cy1lYXN0LTE6MTc1Njg4ODI2ODM4NTpHOkw4eTNHTHBN', 'content-type': 'text/xml', 'content-length': '451', 'date': 'Wed, 03 Sep 2025 08:31:08 GMT'}, 'RetryAttempts': 0}}


25/09/03 08:31:10 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


+----------+
|brand_name|
+----------+
|     3bros|
|      aadi|
|      aayu|
|      acer|
|  achintya|
+----------+
only showing top 5 rows



In [1]:
from pyspark.sql import SparkSession
import os
spark = SparkSession.builder \
    .appName("BigCSVWrite") \
    .config("spark.driver.memory", "5g") \
    .config("spark.executor.memory", "4g") \
    .config("spark.sql.shuffle.partitions", "200") \
    .getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/03 12:41:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame  # or: from pyspark.sql.dataframe import DataFrame
from pyspark.sql import functions as F, types as T

In [34]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

# Should not raise:
spark.sparkContext._jvm.java.lang.Class.forName("org.postgresql.Driver")
ip='172.31.19.202'
db='amazon_sp_api'
password='Techblooprint123'
user="blooprint"

URL  = os.getenv("PG_JDBC_URL", f"jdbc:postgresql://{ip}:5433/{db}")
USER = os.getenv("PG_USER", user)
PWD  = os.getenv("PG_PASSWORD", password)
portfolio_query = "(SELECT * FROM public.portfolio_id_name_mapping) AS p"

portfolio_id_name_mapping = (spark.read.format("jdbc")
      .option("url", URL)
      .option("user", USER)
      .option("password", PWD)
      .option("driver", "org.postgresql.Driver")
      .option("dbtable", portfolio_query)
      .load())
campaign_query = "(SELECT * FROM public.campaigns) as c"
campaigns = (spark.read.format("jdbc")
      .option("url", URL)
      .option("user", USER)
      .option("password", PWD)
      .option("driver", "org.postgresql.Driver")
      .option("dbtable", campaign_query)
      .load())
competitor_query = "(SELECT * FROM public.competitor_brand_data) as cb"
competitors = (spark.read.format("jdbc")
      .option("url", URL)
      .option("user", USER)
      .option("password", PWD)
      .option("driver", "org.postgresql.Driver")
      .option("dbtable", competitor_query)
      .load())
sb_kw_query = "(SELECT * FROM public.sponsored_brands_search_term) as sb"
sb_kw = (spark.read.format("jdbc")
      .option("url", URL)
      .option("user", USER)
      .option("password", PWD)
      .option("driver", "org.postgresql.Driver")
      .option("dbtable", sb_kw_query)
      .load())
sp_kw_query = "(SELECT * FROM public.sponsored_products_search_term) as sp"
sp_kw = (spark.read.format("jdbc")
      .option("url", URL)
      .option("user", USER)
      .option("password", PWD)
      .option("driver", "org.postgresql.Driver")
      .option("dbtable", sp_kw_query)
      .load())
sd_kw_query = "(SELECT * FROM public.sponsored_display_targeting) as sd"
sd_kw = (spark.read.format("jdbc")
      .option("url", URL)
      .option("user", USER)
      .option("password", PWD)
      .option("driver", "org.postgresql.Driver")
      .option("dbtable", sd_kw_query)
      .load())

targets_query = "(SELECT * FROM public.targets ) as t"
targets = (spark.read.format("jdbc")
      .option("url", URL)
      .option("user", USER)
      .option("password", PWD)
      .option("driver", "org.postgresql.Driver")
      .option("dbtable", targets_query)
      .load())
bid_query="""(SELECT unnamed_subquery.keyword_id,
            unnamed_subquery.bid,
            unnamed_subquery.bid_tym
           FROM ( SELECT bid_values.keyword_id,
                    bid_values.bid,
                    bid_values.updated_date::date AS bid_tym,
                    rank() OVER (PARTITION BY bid_values.keyword_id ORDER BY bid_values.updated_date DESC) AS rank
                   FROM bid_values) unnamed_subquery
          WHERE unnamed_subquery.rank = 1 ) bid"""
bid_df = (spark.read.format("jdbc")
      .option("url", URL)
      .option("user", USER)
      .option("password", PWD)
      .option("driver", "org.postgresql.Driver")
      .option("dbtable", bid_query)
      .load())

In [4]:
campaign_df=campaigns.select(F.col("campaignId").alias("campaign_id"),
                 F.col("portfolioId").alias("portfolioId"),
                 F.col("name").alias("campaign_name"),
                 ).dropDuplicates(['campaign_id'])


In [5]:
portfolio_df=portfolio_id_name_mapping.select(F.col("portfolioId").alias("portfolioId"),
                                 F.col("name").alias("portfolio_name")).dropDuplicates(['portfolioId'])
portfolio_mapping=campaign_df.join(portfolio_df,on=campaign_df['portfolioId']==portfolio_df['portfolioId'],how='left')

In [6]:
sb_search_term=sb_kw.select(F.col("account_name"),
             F.split(F.col("campaignId").cast("string"),"\\.").getItem(0).alias("campaignId"),
             F.col("campaignName").alias("campaign_name"),
             F.split(F.col("adGroupId").cast("string"),"\\.").getItem(0).alias("ad_group_id"),
             F.col("adGroupName").alias("ad_group_name"),
             F.split(F.col("keywordId").cast("string"),"\\.").getItem(0).alias("keyword_id"),
             F.col("keywordText").alias("keyword"),
             F.col("searchTerm").alias("search_term"),
             F.col("date").cast("date").alias("date"),
             F.col("impressions").alias("impressions"),
             F.col("clicks").alias("clicks"),
             F.col("cost").alias("ad_spend"),
             F.col("sales").alias("ads_sale"),
             F.col("unitsSold").alias("ads_units"),
             F.col("matchType").alias("match_type"),


             )

In [7]:
sd_search_term=sd_kw.select(F.col("account_name"),
             F.split(F.col("campaignId").cast("string"),"\\.").getItem(0).alias("campaignId"),
             F.col("campaignName").alias("campaign_name"),
             F.split(F.col("adGroupId").cast("string"),"\\.").getItem(0).alias("ad_group_id"),
             F.col("adGroupName").alias("ad_group_name"),
             F.split(F.col("targetingId").cast("string"),"\\.").getItem(0).alias("keyword_id"),
             F.col("targetingText").alias("keyword"),
             F.lit("not_applicable").alias("search_term"),
             F.col("date").cast("date").alias("date"),
             F.col("impressions").alias("impressions"),
             F.col("clicks").alias("clicks"),
             F.col("cost").alias("ad_spend"),
             F.col("sales").alias("ads_sale"),
             F.col("unitsSold").alias("ads_units"),
             F.lit("DSP TGT").alias("match_type"),

             )

In [8]:
sp_search_term=sp_kw.select(F.col("account_name"),
             F.split(F.col("campaignId").cast("string"),"\\.").getItem(0).alias("campaignId"),
             F.col("campaignName").alias("campaign_name"),
             F.split(F.col("adGroupId").cast("string"),"\\.").getItem(0).alias("ad_group_id"),
             F.col("adGroupName").alias("ad_group_name"),
             F.split(F.col("keywordId").cast("string"),"\\.").getItem(0).alias("keyword_id"),
             F.col("targeting").alias("keyword"),
             F.col("searchTerm").alias("search_term"),
             F.col("date").cast("date").alias("date"),
             F.col("impressions").alias("impressions"),
             F.col("clicks").alias("clicks"),
             F.col("cost").alias("ad_spend"),
             F.col("sales30d").alias("ads_sale"),
             F.col("unitsSoldSameSku1d").alias("ads_units"),
             F.col("matchType").alias("match_type"),


             )

In [9]:
keyword_df=sp_search_term.unionByName(sb_search_term).unionByName(sd_search_term)

In [10]:
keyword_df=keyword_df.withColumn(
    "new_match_type",
    F.when(
        (F.col("match_type") == "TARGETING_EXPRESSION") & F.col("keyword").like("%asin=%"),
        F.lit("PT EXACT")
    ).when(
        (F.col("match_type") == "TARGETING_EXPRESSION") & F.col("keyword").like("%category=%"),
        F.lit("PT CAT")
    ).when(
        (F.col("match_type") == "TARGETING_EXPRESSION") & F.col("keyword").like("%asin-expanded=%"),
        F.lit("PT EXP")
    ).when(
        F.col("match_type") == "TARGETING_EXPRESSION_PREDEFINED",
        F.upper(F.col("keyword"))
    ).otherwise(F.col("match_type"))
)

In [11]:
our_asins_df=competitors.select(
    F.col("our_asins"),
    F.col("account_name")
).where(
    F.col("our_asins").isNotNull()

).dropDuplicates()

In [12]:
keyword_df=keyword_df.withColumn(
    "asin_extracted",
    F.when(
        (F.col("keyword").contains("asin=")) | (F.col("keyword").contains("asin-expanded=")),
        # split on "=" then take the right side
        F.split(F.col("keyword"), "=").getItem(1)   
    ).otherwise(F.lit(None))
)

In [13]:
from pyspark.sql.functions import broadcast

In [14]:
keyword_df=keyword_df.withColumn("asin_extracted",F.translate(F.col("asin_extracted"),'"',""))

In [15]:
our_asins_df = our_asins_df.withColumnRenamed("account_name", "asin_account_name")

keyword_df = keyword_df.join(
    F.broadcast(our_asins_df),
    (keyword_df["account_name"] == our_asins_df["asin_account_name"]) &
    (keyword_df["asin_extracted"] == our_asins_df["our_asins"]),
    how="left"
)


In [16]:
keyword_df=keyword_df.drop('asin_account_name')

In [17]:

keyword_df.withColumn(
    "isbdasin",
    F.when(F.col("asin_extracted") == F.col("our_asins"), F.lit(True))
     .otherwise(F.lit(False))
)



DataFrame[account_name: string, campaignId: string, campaign_name: string, ad_group_id: string, ad_group_name: string, keyword_id: string, keyword: string, search_term: string, date: date, impressions: decimal(20,2), clicks: decimal(20,2), ad_spend: decimal(20,2), ads_sale: decimal(20,2), ads_units: decimal(20,2), match_type: string, new_match_type: string, asin_extracted: string, our_asins: string, isbdasin: boolean]

In [18]:
self_brand_name=competitors.select(
    F.col("our_brand_name"),
    F.col("account_name")
).dropDuplicates()

In [19]:
self_brand_name=self_brand_name.filter(
    (F.col("our_brand_name").isNotNull())|(F.col("our_brand_name")!="")

)

In [20]:
self_brand_name = (
    self_brand_name
    .withColumn(
        "our_brand_name",
        F.regexp_replace("our_brand_name", "Your Everyday Greens", "Everyday")
    )
    .withColumn(
        "our_brand_name",
        F.regexp_replace("our_brand_name", "Beyond Appliances", "Beyond")
    )
)

In [21]:
brand_regex = (
    self_brand_name
      .groupBy("account_name")
      .agg(
          F.concat_ws("|", F.collect_set("our_brand_name")).alias("brand_pattern")
      )
      .withColumn("brand_pattern", F.concat(F.lit("(?i)("), F.col("brand_pattern"), F.lit(")")))
)

In [22]:

keyword_df = (
    keyword_df
    .join(F.broadcast(brand_regex), on="account_name", how="left")
    .withColumn("bd_brand",
        F.expr("keyword rlike brand_pattern")
    )
)

In [23]:
comp_brand_name=competitors.select(
    F.col("competitor_brand_name"),
    F.col("account_name")
).dropDuplicates()

In [24]:
comp_brand_name=comp_brand_name.filter(
    (F.col("competitor_brand_name").isNotNull())|(F.col("competitor_brand_name")!="")
).dropDuplicates()

In [25]:
comp_brand_name_regex=comp_brand_name.groupby("account_name").agg(F.concat_ws("|",F.collect_set("competitor_brand_name")).alias("competitor_brand_name"))

In [26]:
keyword_df=(
    keyword_df
    .join(F.broadcast(comp_brand_name_regex), on="account_name", how="left")
    .withColumn("ba_brand", F.expr("lower(keyword) rlike lower(competitor_brand_name)"))

)

In [27]:
keyword_df=keyword_df.withColumn(
    "strategy",
    F.when(
        (F.col("asin_extracted") == F.col("our_asins")) | F.col("bd_brand"),
        "bd"
    )
    .when(
        F.col("ba_brand") |
        (
            (F.col("asin_extracted").isNotNull()) &
            (
                (F.col("asin_extracted") != F.col("our_asins")) |
                (F.col("our_asins").isNull())
            )
        ),
        "ba"
    )
    .otherwise("generic")
)

In [28]:
keyword_df=keyword_df.drop("asin_extracted","our_asins","brand_pattern",'bd_brand',"ba_brand","competitor_brand_name")

In [29]:

keyword_df=keyword_df.select('account_name',
 'campaign_name',
 'ad_group_id',
 'ad_group_name',
 'keyword_id',
 'keyword',
 'search_term',
 'date',
 'impressions',
 'clicks',
 'ad_spend',
 'ads_sale',
 'ads_units',
 'match_type',
 'new_match_type',
 'strategy')

In [30]:
from pyspark.sql import functions as F

target_df = (
    targets
    .select(
        F.col("campaignId").alias("campaign_id"),
        F.col("targetType").alias("targeting_type"),
        F.col("targetDetails_event").alias("target_event")
    )
    .withColumn(
        "target_group",
        F.when(F.upper(F.col("targeting_type")) == "PRODUCT", "PT")
         .when(F.upper(F.col("targeting_type")) == "KEYWORD", "KT")
         .when(F.upper(F.col("targeting_type")).isin("AUTO", "THEME"), "AT")
         .when(F.upper(F.col("targeting_type")) == "PRODUCT_CATEGORY", "CT")
         .when(F.upper(F.col("targeting_type")) == "AUDIENCE", "ADT")
         .when(
             (F.upper(F.col("targeting_type")).isin("PRODUCT_CATEGORY_AUDIENCE", "PRODUCT_AUDIENCE")) &
             (F.upper(F.col("target_event")) == "PURCHASES"),
             "PR"
         )
         .when(
             (F.upper(F.col("targeting_type")).isin("PRODUCT_CATEGORY_AUDIENCE", "PRODUCT_AUDIENCE")) &
             (F.upper(F.col("target_event")) == "VIEWS"),
             "VR"
         )
         .otherwise("Not Applicable")
    )
    .dropDuplicates()
)
